## Trabalho ED2 - Grafos neo4j

## Nome: Victor Misael Brito Fenelon Carneiro
## Matrícula:564227

In [1]:
!pip install scipy
!pip install tqdm
!pip install pandas
!pip install neo4j
!pip install matplotlib
!pip install networkx


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importa as libs necessárias

In [2]:
import pandas as pd


#Biblioteca para operações com expressões regulares (regex)
import re

#Biblioteca para emoji
import emoji

In [3]:
# Função para mostrar estatísticas de antes e depois de cada filtragem
def display_counts(df, stage_name):
    print(f"\n### {stage_name} ###")
    print(f"Quantidade de mensagens: {len(df)}")

In [4]:


df = pd.read_csv("tb_whatsapp_messages.csv")



C:\Users\Misael\AppData\Local\Temp\ipykernel_13504\2954495583.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("tb_whatsapp_messages.csv")


## Remover trava zaps

In [5]:

pattern = r'(.)\1{100,}'  # Padrão para caracteres repetidos


if 'text_content_anonymous' in df.columns:
    mask = df['text_content_anonymous'].apply(lambda x: bool(re.search(pattern, str(x))))
    
    count_incorrect_lines = mask.sum()
    
    print("Número total de linhas com caracteres incomuns:", count_incorrect_lines)
    
    
    incorrect_lines = df[mask]
    
    
    df = df[~mask]
else:
    print("A coluna 'text_content_anonymous' não existe no DataFrame.")


Número total de linhas com caracteres incomuns: 192


## Removendo textos curtos

In [6]:
 df = df[  df['text_content_anonymous'].apply(lambda x: isinstance(x, str) and len(x.split()) >= 5)]


## Pre processa os dados

In [7]:
df_messages = df[['id_member_anonymous', 'id_group_anonymous', 'id','score_misinformation',"text_content_anonymous"]]
df_messages.rename(columns={'id_member_anonymous': 'sender', 'id_group_anonymous': 'group_id', 'id': 'message_id'}, inplace=True)
## Faz uma relação entre receptores e quem enviou mensagens
df_members = df[['id_member_anonymous', 'id_group_anonymous']].drop_duplicates()
df_members.rename(columns={'id_member_anonymous': 'receiver', 'id_group_anonymous': 'group_id'}, inplace=True)

df_merged = df_messages.merge(df_members, on='group_id')
# Talvez nem precise dessa parte a baixo, ja que ele tb "recebe" a mensagem na qual enviou
df_merged = df_merged[df_merged['sender'] != df_merged['receiver']]

C:\Users\Misael\AppData\Local\Temp\ipykernel_13504\176719007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages.rename(columns={'id_member_anonymous': 'sender', 'id_group_anonymous': 'group_id', 'id': 'message_id'}, inplace=True)


### LIsta de usuários

In [8]:
df_nodes = df[["id_member_anonymous"]].drop_duplicates()

## Lista de grupos

In [9]:
df_grpos = df[["id_group_anonymous"]].drop_duplicates()

## Cria o grafo geral de mensagens

In [10]:
# Simplesmente agrega receptores e quem envia as mensagens
df_general_edges = df_merged.groupby(['sender', 'receiver']).size().reset_index(name='weight')

## Cria arestas virais

In [11]:
# Conta o número de vezes que essa mensagem foi enviada
viral_messages = df.groupby('text_content_anonymous').size().reset_index(name='sent_count')
# Filtra aquelas enviadas mais de 100 vezes
viral_messages = viral_messages[viral_messages["sent_count"]>1]

viral_messages = viral_messages.sort_values(by="sent_count")

# Filtra todas as mensagens enviadas somente por mensagens virais

df_viral = df_merged[df_merged['text_content_anonymous'].isin(viral_messages['text_content_anonymous'])]

# Todas as mensagens de df_viral são virais
# mensagem viral para um grupo do qual o usuário j faz parte e o peso dessa aresta é 
# quantidade de mensagens virais enviadas pelo usuário i para aquele grupo. 
df_edges_viral = df_viral.groupby(['sender', 'receiver', 'group_id']).size().reset_index(name='viral_message_count')

# Passo 4: Agregar o peso por sender e receiver (soma total das mensagens virais entre os usuários)
df_edges_viral = df_edges_viral.groupby(['sender', 'receiver']).agg(
    weight=('viral_message_count', 'sum')  # Soma das mensagens virais
).reset_index()



## cria arestas de desinformação

In [12]:
# Filtrar mensagens que contenham desinformação
df_misinformation = df_merged[df_merged['score_misinformation'] > 0.7]

# Criar arestas entre sender e receiver baseadas nos grupos compartilhados
# Agrupamos por sender, receiver, e group_id para calcular a soma das mensagens de desinformação enviadas
df_misinformation = df_misinformation.groupby(['sender', 'receiver', 'group_id']).size().reset_index(name='misinformation_message_count')

# Passo 3: Agregar o peso por sender e receiver (soma total das mensagens de desinformação entre os usuários)
df_misinformation = df_misinformation.groupby(['sender', 'receiver']).agg(
    weight=('misinformation_message_count', 'sum')  # Soma das mensagens de desinformação
).reset_index()

In [13]:
# display(len(df_edges_misinformation))
display(len(df_general_edges))
display(len(df_edges_viral))
display(len(df_misinformation))

964630

512353

242276

In [14]:
import networkx as nx
def count_nodes_and_edges(df):
    """
    Conta o número de nós e arestas em uma tabela pandas.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os campos 'sender', 'receiver', e 'weight'.
    
    Retorna:
        tuple: Um tuplo com (numero_de_nos, numero_de_arestas).
    """
    # Contar os nós únicos no campo 'sender' e 'receiver'
    unique_nodes = df['sender'].drop_duplicates()
    number_of_nodes = len(unique_nodes)
    
    # O número de arestas é o número de linhas da tabela
    number_of_edges = len(df)
    
    return number_of_nodes, number_of_edges


In [15]:
display(count_nodes_and_edges(df_general_edges))
display(count_nodes_and_edges(df_edges_viral))
display(count_nodes_and_edges(df_misinformation))

(6120, 964630)

(3455, 512353)

(1634, 242276)

In [16]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"  # Replace with your URI
username = "neo4j"
password = "test1234567"

driver = GraphDatabase.driver(uri, auth=(username, password))

from tqdm import tqdm

def create_user_index():
    """
    Create an index on the id_usuario property for Usuario nodes.
    """
    query = """
    CREATE INDEX IF NOT EXISTS  FOR (n:Usuario) ON (n.id_usuario);

    """
    with driver.session() as session:
        session.run(query)
        print("Index on Usuario(id_usuario) created successfully.")
        
def create_grupo_index():
    """
    Create an index on the id_usuario property for Usuario nodes.
    """
    query = """
    CREATE INDEX IF NOT EXISTS  FOR (n:Grupo) ON (n.id_grupo);

    """
    with driver.session() as session:
        session.run(query)
        print("Index on Usuario(id_usuario) created successfully.")

def batch_create_with_unwind(session, query, data, batch_size=1000, desc=""):
    """
    Execute Cypher queries using UNWIND for batch inserts.
    """
    for i in tqdm(range(0, len(data), batch_size), desc=desc, unit="batch"):
        batch = data[i:i + batch_size]
        with session.begin_transaction() as tx:
            tx.run(query, batch=batch)
            tx.commit()

def create_nodes(df, batch_size=1000):
    """
    Create nodes in batches using UNWIND.
    """
    query = """
    UNWIND $batch AS row
    MERGE (n:Usuario {id_usuario: row.id_member_anonymous})
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating Nodes")

def create_nodes_grupos(df, batch_size=1000):
    """
    Create nodes in batches using UNWIND.
    """
    query = """
    UNWIND $batch AS row
    MERGE (n:Grupo {id_grupo: row.id_group_anonymous})
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating Nodes")

def create_message_to_groups(df,batch_size=1000):
    """
    Create general relationships in batches using UNWIND.
    """
    query = """
        UNWIND $batch AS row
        MATCH (s:Usuario {id_usuario: row.id_member_anonymous})
        MATCH (r:Grupo {id_grupo: row.id_group_anonymous})
        MERGE (s)-[rel:ENVIOU_PARA_GRUPO {weight: row.weight}]->(r)
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating Grupo Relationships")


def create_general_relationships(df, batch_size=5000):
    """
    Create general relationships in batches using UNWIND.
    """
    query = """
    UNWIND $batch AS row
    MATCH (s:Usuario {id_usuario: row.sender})
    MATCH (r:Usuario {id_usuario: row.receiver})
    MERGE (s)-[rel:ENVIOU_PARA {weight: row.weight}]->(r)
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating General Relationships")

def create_viral_relationships(df, batch_size=5000):
    """
    Create viral relationships in batches using UNWIND.
    """
    query = """
    UNWIND $batch AS row
    MATCH (s:Usuario {id_usuario: row.sender})
    MATCH (r:Usuario {id_usuario: row.receiver})
    MERGE (s)-[rel:MENSAGEM_VIRAL {weight: row.weight}]->(r)
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating Viral Relationships")


def create_misinformation_relationships(df, batch_size=5000):
    """
    Create viral relationships in batches using UNWIND.
    """
    query = """
    UNWIND $batch AS row
    MATCH (s:Usuario {id_usuario: row.sender})
    MATCH (r:Usuario {id_usuario: row.receiver})
    MERGE (s)-[rel:MISINFORMATION {weight: row.weight}]->(r)
    """
    with driver.session() as session:
        data = df.to_dict('records')
        batch_create_with_unwind(session, query, data, batch_size=batch_size, desc="Creating Misinformation Relationships")



In [17]:
create_user_index()
create_grupo_index()
create_nodes(df_nodes)
create_nodes_grupos(df_grpos)
create_message_to_groups(df.groupby(['id_member_anonymous', 'id_group_anonymous']).size().reset_index(name='weight'))

Index on Usuario(id_usuario) created successfully.
Index on Usuario(id_usuario) created successfully.


Creating Grupo Relationships: 100%|███████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.59batch/s]


In [18]:
create_viral_relationships(df_edges_viral)
create_general_relationships(df_general_edges )
create_misinformation_relationships(df_misinformation)

Creating Misinformation Relationships: 100%|████████████████████████████████████████| 49/49 [00:12<00:00,  3.92batch/s]


In [19]:
def create_graph(dataset):
    # Criar o grafo orientado
    graph = nx.DiGraph()

    # Adicionar arestas com pesos
    for _, row in dataset.iterrows():
        graph.add_edge(row['sender'], row['receiver'], weight=row['weight'])

    return graph


g1 = create_graph(df_general_edges)
g2 = create_graph(df_edges_viral)
g3 = create_graph(df_misinformation)

In [20]:
def compute_metrics(graph, name):
    """
    Computa métricas de centralidade e propriedades globais do grafo.
    
    Parâmetros:
        graph (networkx.Graph): O grafo analisado.
        name (str): Nome do grafo para exibição.
    """
    # Centralidades
    degree_centrality = nx.degree_centrality(graph)
    closeness_centrality = nx.closeness_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph, normalized=True)
    clustering_coefficient = nx.clustering(graph)


    return degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient


In [21]:
c1 = compute_metrics(g1, "Mensagem em Geral")
c2 = compute_metrics(g2, "Mensagem Viral")
c3 = compute_metrics(g3, "Mensagem com Desinformação")

In [53]:
def create_dfs(c1,name):
    degree_centrality_c1= pd.DataFrame({'user_id': list(c1[0].keys()), 'degree_centrality': list(c1[0].values())})
    closeness_centrality_c1=pd.DataFrame({'user_id': list(c1[1].keys()), 'closeness_centrality': list(c1[1].values())})
    betweenness_centrality_c1=pd.DataFrame({'user_id': list(c1[2].keys()), 'betweenness_centrality': list(c1[2].values())})
    clustering_coefficient_c1=pd.DataFrame({'user_id': list(c1[3].keys()), 'clustering_coefficient': list(c1[3].values())})

       # Save the DataFrames as CSV files
    degree_centrality_c1.to_csv(f"{name}_degree_centrality.csv", index=False)
    closeness_centrality_c1.to_csv(f"{name}_closeness_centrality.csv", index=False)
    betweenness_centrality_c1.to_csv(f"{name}_betweenness_centrality.csv", index=False)
    clustering_coefficient_c1.to_csv(f"{name}_clustering_coefficient.csv", index=False)

In [54]:
create_dfs(c1,"geral")
create_dfs(c1,"viral")
create_dfs(c1,"desinformacao")


In [55]:
def load_dfs(name):
    """
    Loads DataFrames from CSV files.

    Args:
        name: The base filename of the CSV files.

    Returns:
        A tuple of DataFrames: (degree_centrality, closeness_centrality, betweenness_centrality, clustering_coefficient)
    """

    degree_centrality = pd.read_csv(f"{name}_degree_centrality.csv")
    closeness_centrality = pd.read_csv(f"{name}_closeness_centrality.csv")
    betweenness_centrality = pd.read_csv(f"{name}_betweenness_centrality.csv")
    clustering_coefficient = pd.read_csv(f"{name}_clustering_coefficient.csv")

    return degree_centrality, closeness_centrality, betweenness_centrality, clustering_coefficient



In [56]:
desinf=load_dfs("desinformacao")
viral=load_dfs("viral")
geral=load_dfs("geral")

In [45]:
desinf

{'0000ccace97501f92535d131962ea97d': 0.004418262150220913,
 '26288cd05a9c9505cbc67490913624a8': 0.0018000327278677793,
 '3409853e35408827a292026277c823c0': 0.0018000327278677793,
 '621d3b49685e82eb73610df3653a6d17': 0.004418262150220913,
 '736e4425e6cd37e7ed441642a324cbdd': 0.004418262150220913,
 '79b47ea3c3bc2f62d8aec39850039815': 0.004418262150220913,
 '84865d2dc2e4f8f9e95fea3309d12ebe': 0.004418262150220913,
 '90276bea72f6c8e2ec793e5542578c85': 0.0018000327278677793,
 '91d802b9f81c45c9ced7a96aa187b4e8': 0.004418262150220913,
 '9e7912a0dc38b121d37a7285354d5ca1': 0.0018000327278677793,
 'a5c3b188a3fba948f1778b2a6cf47912': 0.004418262150220913,
 'b40a51577c2b9e6584dbc7e221c6d8a4': 0.004418262150220913,
 'bbbead973c720c67caa0bf6aa8ddf579': 0.004418262150220913,
 'c0be09537ab32bc8baf1678f24d546f8': 0.004418262150220913,
 'c34cb39b379c21901365a9c115b4a080': 0.0018000327278677793,
 'd1d0276cacc4382ac1dfd76d97c023fe': 0.004418262150220913,
 'd1e9eef3c4444f9eeacc992328ccf130': 0.001800032727